##### 问题:
当创建类实例时我们想返回一个缓存引用，让其指向上一个用同样参数（如果有的话）
创建出的类实例。

##### 解决方案:
本节提到的这个问题常常出现在当我们想确保针对某一组输入参数只会有一个类实例
存在时。现实中的例子包括一些库的行为，比如在 logging 模块中，给定的一个名称只
会关联到一个单独的 logger 实例。示例如下：

In [36]:
import logging
a = logging.getLogger('foo')
b = logging.getLogger('bar')
print(a is b)

c = logging.getLogger('foo')
a is c

False


True

要实现这一行为，应该使用一个与类本身相分离的工厂函数。示例如下：

In [37]:
# The class in question
class Spam:
    def __init__(self, name):
        self.name = name
# Caching support
import weakref                                      #弱链接?
_spam_cache = weakref.WeakValueDictionary()
def get_spam(name):
    if name not in _spam_cache:
        s = Spam(name)                              #创建一个实例
        _spam_cache[name] = s                       #存入字典
    else:
        s = _spam_cache[name]                       #存入字典
    return s

如果你用上述实现，会发现 Spam 类的行为和之前展示的效果一样：


In [38]:
a = get_spam('foo')
b = get_spam('bar')
print(a is b)

c = get_spam('foo')
print(a is c)


False
True


要想修改实例创建的规则，编写一个特殊的工厂函数常常是一种简单的方法。此时，
一个常被提到的问题就是是否可以用更加优雅的方式来完成呢？

例如，我们可能会考虑重新定义类的__new__()方法：

In [39]:
# Note: This code doesn't quite work
import weakref
class Spam:
    _spam_cache = weakref.WeakValueDictionary()
    def __new__(cls, name):
        if name in cls._spam_cache:
            return cls._spam_cache[name]
        else:
            self = super().__new__(cls)
            cls._spam_cache[name] = self
            return self
    def __init__(self, name):
        print('Initializing Spam')
        self.name = name

初看上去，上面的代码似乎可以完成任务。但是，主要的问题在于__init__()方法总是
会得到调用，无论对象实例有无得到缓存都是如此。示例如下：

In [40]:
s = Spam('Dave')
t = Spam('Dave')
s is t

Initializing Spam
Initializing Spam


True

这种行为很可能不是我们想要的。因此，要解决实例缓存后会重复初始化的问题，需
要采用一个稍有些不同的方法。

本节中对弱引用的运用与垃圾收集有着极为重要的关系。当维护实例缓存时，只要在
程序中实际用到了它们，那么通常希望将对象保存在缓存中。WeakValueDictionary 会
保存着那些被引用的对象，只要它们存在于程序中的某处即可。否则，当实例不再被
使用时，字典的键就会消失。示例如下：

In [41]:
a = get_spam('foo')
b = get_spam('bar')
c = get_spam('foo')
print(list(_spam_cache))
del a
del c
print(list(_spam_cache))
del b
list(_spam_cache)

['foo', 'bar']
['bar']


[]

对于许多程序而言，使用本节中给出的框架代码通常就足够了。但是，还可以考虑一
些更加高级的实现技术。

我们立刻能想到的是，本节中的解决方案需要依赖全局变量以及一个与原始的类定义
相分离的工厂函数。一种改进方式是将缓存代码放到另一个单独的管理类中，然后将
这些组件粘合在一起：

In [42]:
import weakref
class CachedSpamManager:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()  #这个字典很重要

    def get_spam(self, name):
        if name not in self._cache:
            s = Spam(name)
            self._cache[name] = s
        else:
            s = self._cache[name]
        return s

    def clear(self):
        self._cache.clear()

class Spam:
    manager = CachedSpamManager()
    def __init__(self, name):
        self.name = name

    def get_spam(name):
        return Spam.manager.get_spam(name)

这种方法的特点就是为潜在的灵活性提供了更多支持。例如，我们可以实现不同类型
的缓存管理机制（以单独的类来实现），然后附加到 Spam 类中替换掉默认的缓存实现。
其他的代码（比如 get_spam）不需要修改就能正常工作。

另一种设计上的考虑是到底要不要将类的定义暴露给用户。如果什么都不做的话，用
户可以很容易创建出实例，从而绕过缓存机制：

In [43]:
a = Spam('foo')
b = Spam('foo')
a is b

False

如果预防出现这种行为对程序而言很重要，我们可以采取特定的步骤来避免。例如，
可以在类名前加一个下划线，例如_Spam，这样至少可以提醒用户不应该直接去访
问它。


或者，如果想为用户提供更强的提示，暗示他们不应该直接实例化 Spam 对象，可以让
\_\_init\_\_()方法抛出一个异常，然后用一个类方法来实现构造函数的功能，就像下面
这样：

In [ ]:
class Spam:
    def __init__(self, *args, **kwargs):
        raise RuntimeError("Can't instantiate directly")

 # Alternate constructor
    @classmethod
    def _new(cls, name):      #写一个new方法来进行实例化对象
        self = cls.__new__(cls)
        self.name = name 

要使用上述代码，可以将实现缓存机制的代码修改为使用 Spam._new()来创建实例，而
不是使用通常所见的 Spam()。示例如下：

In [ ]:
import weakref
class CachedSpamManager:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()
    def get_spam(self, name):
        if name not in self._cache:
            s = Spam._new(name) # Modified creation
            self._cache[name] = s
        else:
            s = self._cache[name]
        return s 

尽管还有更加极端的方法来隐藏 Spam 类的可见性，但也许最好不要把问题想的过于
复杂。在类名前添加下划线或者用类方法作为构造函数通常就足以给程序员带来提
示了。


通过使用元类，缓存机制以及其他的创建模式（creational pattern）通常能够以更加优
雅的方式得以解决。关于元类，请参阅 9.13 节。